# 显式指定输入和输出State

默认情况下，图的输入模式和输出模式是相同的。如果你想要修改这种默认设置，也可以直接指定显式的输入模式和输出模式。当你的状态包含大量键，且其中一部分键明确用于输入、另一部分明确用于输出时，这种方式会非常实用。在这种情况下，我们会定义一个“完整”模式，其中包含图操作所需的所有键；同时，还会定义输入和输出模式，它们是“完整”模式的子集，用于限制图的输入和输出内容。

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

# 定义输入数据的结构模式
class InputState(TypedDict):
    question: str

# 定义输出数据的结构模式
class OutputState(TypedDict):
    answer: str

# 定义图的完整状态模式，整合了输入和输出
class OverallState(InputState, OutputState):
    pass

# 定义处理输入并生成答案的节点函数
def answer_node(state: InputState):
    # 返回示例答案，以及一个额外的键值（原样返回问题）
    # 即使没有传入OverallState，也能够对answer进行操作
    return {"answer": "bye", "question": state["question"]}

# 构建图，指定输入模式和输出模式
builder = StateGraph(OverallState, input_schema=InputState, output_schema=OutputState)
builder.add_node(answer_node)  # 添加答案生成节点
builder.add_edge(START, "answer_node")  # 定义开始边，指向 answer_node
builder.add_edge("answer_node", END)  # 定义结束边，从 answer_node 指向结束
graph = builder.compile()  # 编译图

# 使用输入数据调用图，并打印结果
result = graph.invoke({"question": "hi"})
# 只有OutputState的字段会被返回
print(result)

{'answer': 'bye'}


# PrivateState

PrivateState是一个独立于全局状态之外的状态通道，是“草稿纸”，只有节点自己能看到，用于处理不想共享的中间结果、敏感信息或复杂的内部逻辑，主要使用在以下场景：
1. 中间计算或敏感数据处理：当节点需要进行复杂的中间计算，或者需要处理一些不应该直接暴露给最终用户或下游其他节点的敏感数据时，可以使用 PrivateState；
2. 节点内部状态管理：某个节点有自己独立的、复杂的内部状态逻辑，且不希望与全局状态（OverallState）耦合过紧；
3. 性能优化（避免大对象序列化）：LangGraph 的 Checkpoint 机制会序列化整个状态。如果某些数据非常大（如加载的文档片段、图像的 Base64 编码），但后续节点并不需要使用，将其放入 PrivateState 并在使用后及时丢弃（或不写入持久化层），可以减小 Checkpoint 的体积，提高性能；
4. 模块化与代码复用：如果你开发的节点是一个通用组件（比如一个通用的搜索封装节点），你希望它能在不同的项目中复用。如果它依赖 OverallState 中有特定的字段，那么复用时就会要求目标项目的 OverallState 必须做相应修改，非常不灵活。

In [ ]:
# 定义输入状态：包含用户输入的内容
class InputState(TypedDict):
    user_input: str

# 定义输出状态：包含图最终输出的内容
class OutputState(TypedDict):
    graph_output: str

# 定义图的完整状态：包含所有中间和最终数据
class OverallState(TypedDict):
    foo: str          # 中间变量
    user_input: str   # 用户输入
    graph_output: str # 图输出

# 定义私有状态：节点内部使用的额外数据通道
class PrivateState(TypedDict):
    bar: str

def node_1(state: InputState) -> OverallState:
    # 写入 OverallState
    return {"foo": state["user_input"] + " name"}

def node_2(state: OverallState) -> PrivateState:
    # 从 OverallState 读取，写入 PrivateState
    return {"bar": state["foo"] + " is"}

def node_3(state: PrivateState) -> OutputState:
    # 从 PrivateState 读取，写入 OutputState
    return {"graph_output": state["bar"] + " Lance"}

# 构建图：指定完整状态、输入模式和输出模式
builder = StateGraph(OverallState, input_schema=InputState, output_schema=OutputState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

# 编译并运行图
graph = builder.compile()
graph.invoke({"user_input": "My"})

{'graph_output': 'My name is Lance'}

以上流程梳理：
1. 检测：LangGraph 看到 node_2 的返回类型是 PrivateState（显式声明返回类型-> PrivateState，建议都这么做，虽然不声明也不会报错）；
2. 合并：LangGraph 发现 PrivateState 中有一个字段 bar，而当前的 OverallState 中没有；
3. 扩展：LangGraph 自动将 bar 字段“合并”进图的总体状态定义中；
4. 结果：图的实际状态现在变成了 {"foo": ..., "bar": ...}。

# 总结

在 LangGraph 中，Input 定义了节点能“读”什么，Output 规定了节点对外“写”什么，而 PrivateState 则实现了状态的“动态扩展”，三者共同协作，在保证类型安全的同时，实现了从数据输入、中间私密计算到最终结果输出的完整闭环管理。

| 维度 | InputState | OutputState | PrivateState |
| :--- | :--- | :--- | :--- |
| **核心作用** | **约束读取** | **过滤输出** | **扩展与隔离** |
| **数据流向** | 流入节点 | 流出节点 | 在节点内部或图内部流转 |
| **是否必须** | 节点需声明才能读 | 可选（默认返回全部） | 可选（按需动态添加） |
| **可见性** | 节点可见 | 外部/下游可见 | 仅拥有该状态的节点可见 |
| **典型场景** | 提供上下文 | 格式化最终答案 | 中间计算、敏感数据、模块化组件 |